# Statistical Power

Author: Greg Damico.

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import itertools

The power of a statistical test is simply P(reject null | null is false). That is, a test's power grows with the chance that we shall accurately reject the null hypothesis.

If $\beta$ is the probability of a false negative, i.e. the probability that we fail to reject the null hypothesis when it is false, then clearly the power of an associated test is simply $1 - \beta$.

## Example of Power Growing with Sample Size
Consider some particular null hypothesis to the effect that Hepomanol, an experimental drug, has no effect in the treatment of liver cancer. And suppose in fact that this hypothesis is false. A typical test for the effectiveness of Hepomanol would involve patients with liver cancer taking both Hepomanol and a placebo. Some patients would show amelioration of symptoms and others would not. Given that the null hypothesis is false, we ought to expect to see a correlation between those who took Hepomanol and those who experienced relief of symptoms. But the probability that we might, by random chance, see _no_ correlation is higher for smaller sample sizes. The more patients we test, the more likely we are to observe the actually existing correlation. Thus, all else equal, tests involving larger samples will have higher degrees of power.

Suppose in particular that we have the following data about our liver cancer patients.

1. The average tumor length for those given a placebo is 5 cm.
2. The tumor lengths for the ten patients given Hepomanol are:

5, 2, 2, 5, 2.5, 2, 2, 3, 2.5, 2.25

Hepomanol has in fact had great success, and so we'll assume that the null hypothesis is in fact false.

Our statistical test will take the form of sampling from this population of ten. We'll see that the larger the sample is, the greater is the power of the test.

We'll be using a t-test here, so we'll need to have access to t-critical values: We'll use scipy.stats.t for those.

We'll also need the standard deviation of our sample.

In [ ]:
# Calculating the power of a statistical test:


data = pd.Series([5, 2, 2, 5, 2.5, 2, 2, 3, 2.5, 2.25])

Suppose we take a sample of 3. In that case, with a one-tail test and a 5% threshold, there are two degrees of freedom. Let's calculate our critical value:

In [ ]:
stats.t.ppf(0.05, 2)

 So our chance of rejecting the null hypothesis is just the chance that our statistic $t\leq -2.92$.

t is calculated as follows: $\Large\frac{\bar{x} - \mu}{\frac{s}{\sqrt{n}}}$.


There are $10\choose 3$ ($=120$) ways of selecting three data points from a population of ten. The loop below counts the number of times that we would reject the null hypothesis. Thus we can calculate the power of our test by simply taking that number and dividing by $120$.

In [ ]:
combs = list(itertools.combinations(data, 3))
t_crit = stats.t.ppf(0.05, 2)
ctr = 0

for comb in combs:
    if np.std(comb) != 0:
        t = (np.mean(comb) - 5) / (np.std(comb) / 3**0.5)
        if t <= t_crit:
            ctr += 1
    else:
        ctr += 1
ctr / 120

Let's write a function that will calculate the statistical power of our test for _any_ sample size and _any_ data. Again we are assuming that the null hypothesis is in fact *false*.

In [ ]:
def stat_power(n, obs=data):
    ctr = 0
    combs = list(itertools.combinations(obs, n))
    for sample in combs:
        if np.std(sample) != 0:
            if (np.mean(sample) - 5) / (np.std(sample) / n ** 0.5)\
            <= -stats.t.ppf(0.95, df=n-1):
                ctr += 1
        else:
            if sample[0] == 5:
                pass
            else:
                ctr += 1
    power = ctr / len(combs)
    return power

In [ ]:
stat_power(3)

In [ ]:
# Plotting

from matplotlib import pyplot as plt
%matplotlib inline

X = range(3, 9)
y = [stat_power(i) for i in X]

fig, ax = plt.subplots()
ax.plot(X, y, 'g^');